In [2]:
import pandas as pd

# Table 7.1

In [221]:
df = pd.read_csv("raw/table-71.csv").fillna(-1).convert_dtypes().rename(columns={
  'opened_ym': 'date',
  'control_fi': 'isControlled',
  'loan_type': 'loanType',
  'grp_age': 'ageGroup',
  'fi_type': 'fiType',
  'flag_urban_2021': 'isUrban',
  'region_code': 'region',
  'flag_newbrw': 'isNewBorrower',
  'sum_curbal': 'amount',
  'num_ac': 'accounts'
})

# use 0 for NaNs in value columns
df['amount'] = df['amount'].replace(-1, 0)
df['accounts'] = df['accounts'].replace(-1, 0)

unique = ['date', 'isControlled']
facets = list(df.columns[2:8])
values = ['amount', 'accounts']
df = df.sort_values(unique)[unique + facets + values]

In [223]:
# one way to do it is split the database into many json files

for f in facets:
  x = unique + [f]
  tmp = df[x + values].groupby(x, dropna=False).sum()
  new_index = pd.MultiIndex.from_product([tmp.index.unique(level=0), tmp.index.unique(level=1), tmp.index.unique(level=2)])
  tmp = tmp.reindex(new_index).fillna(0)
  tmp = tmp.sort_values([f] + unique[::-1]).droplevel(0)
  groupby = [f] + unique[1:]
  tmp = tmp.groupby(groupby).agg(
    amount=('amount', pd.Series.to_list),
    accounts=('accounts', pd.Series.to_list)
  )
  tmp = tmp.reset_index()
  tmp[groupby + ['amount']].rename(columns={'amount': 'value'}).to_json('json/table-71-' + f + '-amount.json', orient="records")
  tmp[groupby + ['accounts']].rename(columns={'accounts': 'value'}).to_json('json/table-71-' + f + '-accounts.json', orient="records")

# Table 8.3 Borrowers

In [36]:
df = pd.read_csv("raw/table-83-borrowers.csv") \
  .fillna(-1) \
  .convert_dtypes().drop(['period', 'filter_cls'], axis=1) \
  .rename(columns={
    'yearmo': 'period',
    'filter_loantype': 'filter_product',
    # 'filter_fi'
    'by_cls': 'class',
    'by_product': 'product',
    'by_fi': 'fi',
    'by_region': 'region',
    'by_urban': 'urban',
    'by_agegrp': 'age',
    'by_postcovidbrw': 'covid',
    'num_brw': 'count',
  })
df['period'] = pd.to_datetime(df['period'], format="%Y%m").dt.to_period('Q')

In [39]:
# recode
# region: unknown from 0 to 99
df['region'] = df['region'].replace(0, 99)
# urban: rural from 0 to 1, urban from 1 to 2, unknown from 9 to 99
df['urban'] = df['urban'].replace([0, 1, 9], [1, 2, 99])
# age: unknown from 0 to 99
df['age'] = df['age'].replace(0, 99)
# covid: before from 0 to 1, after from 1 to 2
df['covid'] = df['covid'].replace([0, 1], [1, 2])

In [40]:
df.to_csv('processed/table-83-borrowers.csv', index=False)

In [52]:
facet = 'age'
filter_product = 0
filter_fi = 0

In [79]:
x = df[(df['filter_product'] == filter_product) & (df['filter_fi'] == filter_fi)][['period', facet, 'count']].groupby(['period', facet]).sum()

In [89]:
tmp = df[(df['filter_product'] == filter_product) & (df['filter_fi'] == filter_fi)][['period', facet, 'count']].groupby(['period', facet]).sum()
first_period = str(tmp.index.get_level_values(0).sort_values()[0])
new_index = pd.MultiIndex.from_product([tmp.index.unique(level=0), tmp.index.unique(level=1)])
tmp = tmp.reindex(new_index).fillna(0)

In [93]:
tmp = df[(df['filter_product'] == filter_product) & (df['filter_fi'] == filter_fi)][['period', facet, 'count']].groupby(['period', facet]).sum()
new_index = pd.MultiIndex.from_product([tmp.index.unique(level=0), tmp.index.unique(level=1)])
tmp = tmp.reindex(new_index).fillna(0)
periods = tmp.index.get_level_values(0).drop_duplicates()
tmp = tmp.sort_values([facet, 'period'])
tmp = tmp.groupby(facet).agg(count=('count', pd.Series.to_list))
tmp['count'].to_dict()

{1: [37360,
  35516,
  34845,
  25102,
  29170,
  32021,
  36207,
  27162,
  36779,
  39723,
  54149,
  37011,
  39414,
  42197,
  52650,
  45675,
  56678,
  63367,
  71670,
  55746,
  62041,
  63416,
  65310,
  46073,
  51475,
  56018,
  59372,
  43689,
  50689,
  56964,
  63016,
  46174,
  55909,
  60207,
  69062,
  53387,
  61710,
  66159,
  74517,
  53291,
  64043,
  74153,
  84229,
  69827,
  46959,
  63826,
  82188,
  83885,
  110234,
  132486,
  123509,
  95699],
 2: [168226,
  143941,
  136182,
  126275,
  135661,
  132690,
  137297,
  123366,
  146847,
  144628,
  203680,
  170292,
  155099,
  151713,
  167144,
  166825,
  184853,
  194244,
  215607,
  210845,
  215944,
  209496,
  216304,
  193846,
  202602,
  204243,
  211743,
  192850,
  196081,
  208229,
  220325,
  188911,
  205482,
  202053,
  222233,
  201062,
  213037,
  213586,
  235621,
  201292,
  224175,
  237070,
  251993,
  244127,
  143834,
  187925,
  235940,
  289142,
  335265,
  353502,
  319557,
  336145],
 

In [109]:
flist = ['class', 'urban', 'age']
fval = [1, 2, 1]

for i, f in enumerate(flist):
  if (i == 0):
    mask = df[f] == fval[i]
  else:
    mask = (mask) & (df[f] == fval[i])

In [115]:
df[mask]['age'].unique()

<IntegerArray>
[1]
Length: 1, dtype: Int64

In [116]:
x = {
  'a': 1,
  'b': 2
}

In [121]:
for i in x.keys():
  print(i)

a
b
